In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
import datetime

/home/xyz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def now():
    return '_'.join(str(datetime.datetime.now()).split())

In [3]:
df_train = pd.read_csv('/home/xyz/Documents/Otto_Group_data/train.csv')
df_test = pd.read_csv('/home/xyz/Documents/Otto_Group_data/test.csv')

In [4]:
def convert_class_label_to_int(class_label):
    return int(class_label[6:]) - 1

In [5]:
def find_means_stds(features_array):
    means = [np.mean(features_array[:, i]) for i in range(features_array.shape[1])]
    stds = [np.std(features_array[:, i]) for i in range(features_array.shape[1])]
    return {'means': means, 'stds': stds}

In [6]:
def z_score_feature(feature_slice, mean, std):
    return (feature_slice - mean) / std

In [7]:

renamed_labels = [convert_class_label_to_int(i) for i in df_train['target'].values]

feature_columns = ['feat_' + str(i + 1) for i in range(93)]
df_train['renamed_labels'] = renamed_labels

In [8]:
mstddict = find_means_stds(df_train[feature_columns].values)

In [9]:
for i in range(93):
    df_train['feat_' + str(i + 1)] = z_score_feature(df_train['feat_' + str(i + 1)].values, mstddict['means'][i],
                                                     mstddict['stds'][i])

In [10]:

for i in range(93):
    df_test['feat_' + str(i + 1)] = z_score_feature(df_test['feat_' + str(i + 1)].values, mstddict['means'][i],
                                                    mstddict['stds'][i])

In [11]:
# rf = AdaBoostClassifier(n_estimators=100)
# clrf = RandomForestClassifier(n_estimators=400, max_features=25)
# clf = AdaBoostClassifier(base_estimator=clrf, n_estimators=15)
# clf2 = GradientBoostingClassifier(n_estimators=250, max_depth=8, max_features=15)

clf = GradientBoostingClassifier(n_estimators=700, max_depth=7, max_features=20, learning_rate=0.03)

In [12]:
clf.fit(df_train[feature_columns].values, df_train['renamed_labels'].values)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.03, loss='deviance', max_depth=7,
              max_features=20, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=700,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [13]:
labels_pr = clf.predict_proba(df_test[feature_columns].values)
labels_pr_tr = clf.predict_proba(df_train[feature_columns].values)

In [15]:

now_name = now()

predict_dict = {'id': df_test['id'].values}
for i in range(9):
    predict_dict['Class_' + str(i + 1)] = labels_pr[:, i]
df_sub = pd.DataFrame(predict_dict)



In [20]:
df_sub.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,49503,0.002642,0.000315,0.000184,0.000274,6.071141e-07,0.993787,0.000666,0.001975,0.000156
1,49504,0.000142,0.000328,0.000170,0.000077,1.237000e-07,0.000995,0.000034,0.998091,0.000162
2,49505,0.877219,0.004759,0.003716,0.000937,2.393694e-04,0.005676,0.023783,0.074960,0.008710
3,49506,0.000002,0.000532,0.000389,0.000019,2.664179e-07,0.000039,0.000103,0.998902,0.000014
4,49507,0.000699,0.631727,0.357930,0.004197,6.739381e-05,0.001699,0.002516,0.000790,0.000375


In [17]:
df_sub.to_csv('/home/xyz/Documents/Otto_Group_data/submission2.csv',index = False)